# Оптимизируем код

Обычно Python работает довольно быстро, однако иногда его скорости не хватает.

В большинстве случаев вы можете ускорить работу, выбрав более качественный алгоритм или структуру данных. 

Идея заключается в том, чтобы знать, где это
сделать. 

Даже опытные программисты ошибаются довольно часто. Вам нужно быть очень осторожными и семь раз отмерить, прежде чем отрезать. 

Это приводит нас к использованию таймеров.

# Измеряем время

Вы уже видели, что функция time модуля time возвращает текущее время в формате epoch как число секунд с плавающей точкой. 

Быстрый способ засечь время — получить текущее время, что-то сделать, получить новое время и вычесть из него
первое. 

Напишем соответствующий код и назовем файл time1.py:

In [8]:
from time import time
t1 = time()
num = 5
num *= 2
print(time() - t1)

0.0009968280792236328


В этом примере мы измеряем время, которое требуется на присвоение значения 5 переменной num и умножение его на 2. 

Этот пример не является реалистичным тестом производительности, это лишь пример того, как замерить время выполнения произвольного кода. 

Попробуйте запустить его несколько раз, чтобы увидеть, что время может варьироваться:

Программа работала две-три миллионные доли секунды. Попробуем выполнить что-то помедленнее, вроде функции sleep. 

Если мы усыпим выполнение на секунду, наш таймер покажет значение, чуть больше секунды. Сохраните файл под
именем time2.py:

In [10]:
from time import time, sleep
t1 = time()
sleep(1.0)
print(time() - t1)

1.0008783340454102


Чтобы быть уверенным в результатах, запустим программу несколько раз:

In [11]:
from time import time, sleep
t1 = time()
sleep(1.0)
print(time() - t1)

1.0160064697265625


Как и ожидалось, программе для работы требуется около секунды. 

Если бы это оказалось не так, то либо нашему таймеру, либо функции sleep() должно было бы стать стыдно.

Существует более удобный способ измерить время выполнения фрагментов кода вроде этого — стандартный модуль timeit (http://bit.ly/py-timeit). 

У него имеется функция с именем, как вы уже догадались, timeit(), которая запустит ваш код заданное количество раз и выведет результаты. 

Ее синтаксис выглядит так: timeit.timeit (код, число, количество_раз).

В примерах этого раздела код должен находиться в кавычках, чтобы он выполнялся не после нажатия кнопки Return, а лишь внутри функции timeit(). 

(В следующем разделе вы увидите, как можно измерить время выполнения некоторой
функции, передав ее имя в функцию timeit().) 

Запустим предыдущий пример и измерим время его выполнения. 

Назовем этот файл timeit1.py:

In [16]:
from timeit import timeit
print(timeit('num = 5; num *= 2', number=5000))

0.00022149999995235703


Запустим его несколько раз:

In [17]:
from timeit import timeit
print(timeit('num = 5; num *= 2', number=5000))

0.0006583000000546235


Опять же эти две строки кода выполняются примерно за две миллионные доли секунды. 

Мы можем использовать аргумент repeat функции repeat() модуля timeit, чтобы выполнить код большее количество раз. Сохраните этот файл под именем timeit2.py:

https://docs.python.org/3/library/timeit.html

In [18]:
from timeit import repeat
print(repeat('num = 5; num *= 2', number=1, repeat=3))


[1.2000000424450263e-06, 4.999999418942025e-07, 4.0000008993956726e-07]


https://stackoverflow.com/questions/56763416/what-is-diffrence-between-number-and-repeat-in-python-timeit

In [19]:
print(repeat('num = 5; num *= 2', number=1, repeat=3))

[2.300000005561742e-06, 8.000001798791345e-07, 5.000001692678779e-07]


In [20]:
print(repeat('num = 5; num *= 2', number=1, repeat=3))

[7.999999525054591e-07, 3.000000106112566e-07, 1.999999312829459e-07]


Первый запуск занял две миллионные доли секунды, а второй и третий прошли быстрее. 

Почему? Для этого может быть много причин. 

Это могло произойти, например, потому, что мы тестировали очень небольшой фрагмент кода и скорость его выполнения зависит от того, что компьютер делал в эти моменты, как система Python оптимизировала вычисления, и от многого другого.

Или же это могла быть случайность. 

Попробуем сделать что-то более реалистичное, чем присвоение переменных и вызов функции sleep(). 

Мы измерим производительность, сравнив эффективность нескольких алгоритмов (программной логики) и структур данных (механизмов хранения).

# Алгоритмы и структуры данных

Дзен Python (http://bit.ly/zen-py) гласит: «Должен существовать один, и желательно только один, очевидный способ сделать это». 

К сожалению, иногда он не является очевидным и вам приходится сравнивать альтернативные варианты. 

Например, что лучше использовать для создания списка: цикл for или включение списка?

И что на самом деле значит «лучше»: быстрее, проще для понимания, менее затратно по ресурсам или более характерно для Python?

В следующем упражнении мы создадим список разными способами, сравнив скорость, читаемость и стиль. 

Перед вами файл time_lists.py:

In [21]:
from timeit import timeit
def make_list_1():
    result = []
    for value in range(1000):
        result.append(value)
        return result
def make_list_2():
    result = [value for value in range(1000)]
    return result
print('make_list_1 takes', timeit(make_list_1, number=1000), 'seconds')
print('make_list_2 takes', timeit(make_list_2, number=1000), 'seconds')

make_list_1 takes 0.0010178999998515792 seconds
make_list_2 takes 0.049275600000100894 seconds


In [22]:
from timeit import timeit
def make_list_1():
    result = []
    for value in range(1000):
        result.append(value)
        return result
def make_list_2():
    result = [value for value in range(1000)]
    return result
print('make_list_1 takes', timeit(make_list_1, number=1000), 'seconds')
print('make_list_2 takes', timeit(make_list_2, number=1000), 'seconds')

make_list_1 takes 0.0010386999999809632 seconds
make_list_2 takes 0.05810599999995247 seconds


В каждой функции мы добавляем в список 1000 элементов и вызываем каждую функцию 1000 раз. 

Обратите внимание на то, что в этом тесте мы вызываем функцию timeit(), передавая ей имя функции в качестве первого аргумента вместо кода.

Давайте ее запустим:

Включение списка отработало как минимум в два раза быстрее, чем добавление элементов в список с помощью функции append(). 

Как правило, включение быстрее, чем создание вручную.

Используйте эти идеи, чтобы сделать свой код быстрее.

# Cython, NumPy и расширения C

Если вы усердно работаете, но все еще не можете достичь необходимой производительности, у вас есть и другие варианты.

Cython (http://cython.org/) — это гибрид языков Python и C, разработанный для преобразования Python: в скомпилированный код языка С внесены некоторые
улучшения производительности. 

Эти аннотации относительно малы, они похожи на объявление типов некоторых переменных, аргументов функций или возвращаемых функциями значений. 

Подобные подсказки сделают научные вычисления,
выполняющиеся в циклах, гораздо быстрее — в 1000 раз. 

Документацию и примеры можете найти в Cython wiki (https://github.com/cython/cython/wiki).

Из приложения В вы можете подробнее узнать о NumPy. 

Это математическая библиотека Python, написанная для ускорения на С.

Многие части Python и его стандартной библиотеки написаны на С для скорости и обернуты кодом на Python для удобства. 

При написании приложений эти приемы доступны и вам. 

Если вы знаете С и Python и действительно хотите, чтобы
ваш код «летал», напишите расширение на языке С — это труднее, но улучшение оправдает затраченные усилия.

# PyPy

Около 20 лет назад, когда язык Java только появился, он был медленным, как шнауцер, больной артритом. 

Но когда он стал дорого стоить компании Sun и прочим,
они вложили миллионы в оптимизацию интерпретатора Java и лежащей в его основе виртуальной машины Java (Java Virtual Machine, JVM), заимствуя приемы из уже существовавших тогда языков Smalltalk и LISP. 

Компания Microsoft также вложила много усилий в оптимизацию своего языка C# и .NET VM.

Языком Python никто не владеет, поэтому никто так сильно не старается сделать его быстрее. 

Вы, возможно, используете стандартную реализацию Python.

Она написана на С и часто называется CPython (не путать с Cython).

Как и языки PHP, Perl и даже Java, Python не компилируется в машинный код, он преобразуется в промежуточный язык (он называется байт-кодом или p-кодом), который затем интерпретирует виртуальная машина.

PyPy (http://pypy.org/) — это новый интерпретатор Python, который пользуется некоторыми приемами, ускорившими язык программирования Java. 

Тесты производительности интерпретатора (http://speed.pypy.org/) показывают, что PyPy в каждом тесте быстрее CPython в среднем в шесть раз и до 20 раз в отдельных случаях.

Он работает сPython 2 и 3. 

Вы можете загрузить его и использовать вместо CPython.

PyPy постоянно улучшается и однажды может заменить CPython. 

Чтобы узнать, подходит ли он вам, посетите его официальный сайт.

https://falconframework.org/

# Управление исходным кодом

Когда вы работаете над небольшой группой программ, то обычно можете отслеживать внесенные собственноручно изменения — до тех пор, пока не сделаете глупую
ошибку и не потеряете несколько дней работы. 

Системы управления исходным кодом защитят ваш код от сил зла в лице вас самих.

Если вы работаете в группе, управление исходным кодом становится необходимостью.

Для этой области было создано множество коммерческих и бесплатных решений.

Наиболее популярными в мире открытого исходного кода (где и живет Python) являются Mercurial и Git.

Они оба являются примерами распределенных систем контроля версий, которые
создают несколько копий репозиториев кода. Ранние системы вроде Subversion
работают на одном сервере.

# Mercurial

Mercurial (http://mercurial.selenic.com/) написан на Python. 

Научиться пользоваться им довольно легко, он имеет множество подкоманд для загрузки кода из репозитория Mercurial, добавления файлов, проверки на наличие изменений и объединения изменений из разных источников. 

bitbucket (https://bitbucket.org/) и другие сайты
(http://bit.ly/merc-host) предлагают бесплатный или коммерческий хостинг.

# Git

Git (http://git-scm.com/) изначально создавался для разработки ядра Linux, но теперь является доминирующим в области открытого исходного кода в целом. 

Он похож на Mercurial, хотя некоторые считают, что обучиться ему сложнее. 

GitHub (http://github.com/) — это самый крупный хостинг для git, содержащий более миллиона репозиториев, но существует и множество других хостов (http://bit.ly/githost-scm).


Отдельные примеры программ из этой книги доступны в публичном репозитории git на GitHub (https://github.com/madscheme/introducing-python). 

Если у вас установлена программа git, вы можете загрузить их с помощью следующей команды:

$ git clone https://github.com/madscheme/introducing-python

Вы также можете загрузить код, нажав на следующие кнопки на странице GitHub:

 Clone in Desktop (Клонировать на Рабочий стол), чтобы открыть версию git, установленную на ваш компьютер;

 Download ZIP (Загрузить архив), чтобы получить архивированную версию программ.

Если у вас нет git, но вы хотите попробовать с ним поработать, прочтите инструкцию по установке (http://bit.ly/git-install). 

Здесь я буду говорить о версии с командной строкой, но вам могут быть интересны сайты вроде GitHub, предоставляющие дополнительные услуги, которые в некоторых случаях использовать было бы проще: git имеет много возможностей, но не всегда интуитивно понятен.

Проведем тест-драйв. Далеко уходить не будем, просто посмотрим, как работают некоторые команды.

Создадим новую папку и перейдем в нее:

git init

git add test.py

git status

git commit -m "simple print program"

git diff

git log test.py